<img src="images/strathclyde_banner.png" width="50%" align="left" style="left">

# RFSoC Mixer Mode Introduction

----

<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

This notebook presents an introduction to the different mixer modes available through the RF Data Converters (RF DCs) on the RFSoC2x2 board.

## Aims
  
* Gain a deeper understanding of the RFSoC Mixer Modes, including Coarse, Fine and Bypass modes
* Utalise the `xrfdc` Python package to investigate the mixer settings available in the base overlay
* Present the data inspection and visualisation of the RF DCs using Plotly, including an exploration of interactive plots.

## Table of Contents
* [Introduction](#introduction)
* [Hardware Setup](#hardware_setup)
* [Software Setup](#software_setup)
* [Introducing the Coarse Mixer](#introducing_coarse_mixer)
    * [Coarse Mixer: fs/4](#coarse_fs_4)
    * [Coarse Mixer: -fs/4](#course_min_fs_4)
    * [Coarse Mixer: fs/2](#coarse_fs_2)
* [Introducing the Fine Mixer](#introducing_fine_mixer)
* [Mixer Bypass](#mixer_bypass)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.3, June 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf)

## Revision History

* v1.0 | 26/02/2021 | First notebook revision.

## Introduction <a class="anchor" id="introduction"></a>

The RF-DAC tiles available on the RFSoC each contain two RF-DAC blocks which can be used to transmit, or Digital Up Convert (DUC) an information signal. This is the process of modulating a low frequency baseband information signal with a high frequency carrier signal. This is performed using Quadrature Amplitude Modulation (QAM) and a mixer is utalised to modulate the baseband information signal. Similarly, the RF-ADCs act as a receiver, performing Digital Down Conversion (DDC) on the received high frequency signal. Once more mixers are used to demodulate the signal to reconstruct the original baseband information signal.

An overview of the mixers available for both the RF-ADC and RF-DAC is shown in [Figure 1](#fig-1). The complex mixer is able to operate in three modes:

* Fine Mixer Mode
* Coarse Mixer Mode
* Bypass Mode

The user is able to select the desired oeprating mode of these mixers. 

<a class="anchor" id="fig-1"></a>
<figure>
<img src='./images/IQ_mixer.png' height='40%' width='40%'/>
    <figcaption><b>Figure 1: I/Q Mixer circuitry within RF Data Converters </b></figcaption>
</figure>

The **Coarse Mixer** component implements both the **Coarse** mode and the **Bypass** mode.

* In **Coarse** mode, the mixer is able to demodulate using only a very restricted set of frequencies, however it is able to operate in this mode with lower power than in Fine mode. This is because sine and cosine waves at these frequencies comprise a very limited set of samples. Three options are available in this mode: $\frac{f_{s}}{4}$, $-\frac{f_{s}}{4}$ and $\frac{f_{s}}{2}$.


* In **Bypass** mode, the signal path simply bypasses the I/Q mixing stage, via the coarse stage, and does not undergo any modulation/demodulation.

Frequencies of $\frac{f_{s}}{4}$ and $-\frac{f_{s}}{4}$ are useful for modulating/demodulating to a quarter of the sampling frequency with absolutely minimal cost. The NCO must generate only four samples per sine wave cycle: 0, +1, 0, -1. It is trivial to store these values and to multiply another signal by them for modulation/demodulating. Multiplying by 0, +1 and -1 is trivial, and therefore the modulator or demodulator architecture can be simplified to a counter and some simple combinatorial logic.

The **Fine Mixer** component implements **Fine** mode.

* In **Fine** mode, the mixer uses an NCO, which can generate any arbitrary frequency between $-\frac{f_{s}}{2}$ and $\frac{f_{s}}{2}$. This is done by setting a step size input to the phase accumulator (automatically calculated in the design tools), the lookup table size and the desired frequency. Optional additional parameters can be set, including a phase adjustment and added dithering.

The mixer operates on the concept of heterodyning. When we mix two signals, we obtain two signals; one at $f_{1} + f_{2}$ and another at $f_{1} - f_{2}$. Generally one of these signals is useful, and the other is filtered out by a lowpass filter.

In this notebook, we will be demonstrating many of the features
mentioned above via a simple loopback example.

## Hardware Setup <a class="anchor" id="hardware_setup"></a>

Your RFSoC2x2 board is dual-channel. Notice that there are only
4 SMAs on your board, labeled DAC1, DAC2, ADC1, and ADC2.
Only two sets of RF ADC and RF DAC channels are accessible on
the RFSoC2x2 board.

You should create the loopback connection using SMA cables
as shown below:

* Channel 0: DAC2 to ADC2
* Channel 1: DAC1 to ADC1

See [Figure 2](#fig-2) below for a demonstration.

<a class="anchor" id="fig-2"></a>
<figure>
<img src='./images/rfsoc2x2_connections.jpg' height='40%' width='40%'/>
    <figcaption><b>Figure 2: RFSoC2x2 in loopback operation </b></figcaption>
</figure>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## Software Setup <a class="anchor" id="software_setup"></a>

**Initialise the Board**

As before, we will be using the RFSoC2x2 base overlay to add custom control logic in the Programmable Logic (PL) to communicate with the RF-ADC or RF-DAC. 

Let's now download the base overlay and initialise the drivers.

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

The RFSoC2x2 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs). The base overlay
has a simple method to initialize these clocks. Run the cell below to set
the LMK and LMX clocks to 122.88MHz and 409.6MHz, respectively.

In [2]:
base.init_rf_clks()

**Python Setup**

For this design we will use [Numpy](https://numpy.org/) for calculations, and a helper file, plots.py for our time and frequency demain plots. 

In [3]:
import numpy as np
import plots

It is worthwhile highlighting the `xrfdc` Python package, which we will be utalising to change mixer settings for the RF-DACs and RF-ADCs.

In [4]:
import xrfdc

We define a number of samples and a sampling frequency we will use throughout this example.

In [5]:
number_samples = 256
sample_frequency = 4096e6

The RFSoC2x2 has two channels available to us, however for simplicity this demonstartion will be focusing on channel 0 for the ADC and DAC. We assign these as *tx_channel* and *rx_channel* to shorten the code going forward.

In [6]:
tx_channel = base.radio.transmitter.channel[0]
rx_channel = base.radio.receiver.channel[0]

We can set the amplitude and enable transmission from the RF-DAC to the receiver in the RF-ADC.

In [7]:
tx_channel.control.gain = 0.5
tx_channel.control.enable = True

**Mixer Setting Defaults**

We can now examine the available Mixer Settings for the RF-DAC and RF-ADC, and view the default settings. The same Mixer options are available for both Data Converters:

* **CoarseMixFreq** - This option is enabled when the mixer is set to Coarse Mode. A user can choose between OFF (0x0), $\frac{f_{s}}{2}$ (0x2), $\frac{f_{s}}{4}$ (0x4), $-\frac{f_{s}}{4}$ (0x8) or BYPASS (0x10).

* **EventSource** - Event source for mixer settings. A user can choose: update after register write (0x0), update using SLICE event source (0x1), update using TILE event source (0x2), update using SYSREF event source (0x3), update using MARKER event source (0x4) or update using PL event source (0x5).

* **FineMixerScale** - NCO output scale for fine mixer mode. A user can choose between: auto update (0x0), set to 1.0 (0x1) or set to 0.7 (0x2). General default is 1.0 for RF-ADC and 0.7 for RF-DAC.

* **Freq** - NCO frequency range for fine mixer mode. Range: -Fs to Fs (MHz).

* **MixerMode** - Mixer mode for fine or coarse mixer. Options are: Mixer Mode OFF (only for fine mixer) (0x0), Complex to Complex (0x1), Complex to Real (0x2) or Real to Complex (0x3). 

* **MixerType** - Indicates coarse or fine mixer. Options are: Coarse mixer (0x1), Fine mixer (0x2), OFF (0x3) or DISABLED (has to be set in hardware design).

* **PhaseOffset** - NCO phase offset. Range: -180 to 180 (Exclusive).

For more detail please see pages 180-182 of the [RF Data Converter Product Guide](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_3/pg269-rf-data-converter.pdf).

From this we can examine the RF-DAC default settings. We can see that CoarseMixFreq is OFF, EventSource is immediate after register write, FineMixerScale is 1.0, Freq is 1024, MixerMode is Complex to Real, MixerType is Fine Mixer and PhaseOffset is 0!

In [8]:
# RF-DAC mixer settings
tx_channel.dac_block.MixerSettings

{'CoarseMixFreq': 0,
 'EventSource': 0,
 'FineMixerScale': 1,
 'Freq': 1023.9999999999927,
 'MixerMode': 2,
 'MixerType': 2,
 'PhaseOffset': 0.0}

We can also examine the RF-ADC defaults. We see that CoarseMixFreq is OFF, EventSource is update using TILE event source, FineMixerScale is 1.0, Freq is -1024, MixerMode is Real to Complex, MixerType is Fine Mixer and PhaseOffset is 0!

In [9]:
# RF-ADC channel 0 mixer settings
rx_channel.adc_block.MixerSettings

{'CoarseMixFreq': 0,
 'EventSource': 2,
 'FineMixerScale': 1,
 'Freq': -1024.0,
 'MixerMode': 3,
 'MixerType': 2,
 'PhaseOffset': 0.0}

**Configure NCO Frequency**

The below function is of key importance in highlighting the operation of the mixer settings and will be used throughout the notebook to update the centre frequency ($f_{c}$) of the RF-ADC and RF-DAC. The NCO can be configured differently depending on the required modulation or demodulation. We wish to avoid the inversion of the original spectrum, and so we set positive NCO frequencies in down conversion when the desired signal is located at an even Nyquist band, or negative NCO frequencies when the desired signal is located at odd Nyquist bands. For up conversion, we set negative NCO frequencies when shifting a signal to even Nyquist bands and positive frequencies when shifting a signal to odd Nyquist bands.

<a class="anchor" id="fig-3"></a>
<figure>
<img src='./images/NyquistZones.png' height='50%' width='50%'/>
    <figcaption><b>Figure 3: NCO Setting Example </b></figcaption>
</figure>

In [10]:
# Function Definition

def configure_mixer_frequency(block, btype='adc', fc=1024):
    zone = block.NyquistZone #find which nyquist zone we are in
    even = True if ((zone % 2) == 0) else False # if remainder after % 2 is zero, we are in an even zone
    # If the zone is even, we need RF-ADC centre frequency to be positive, in an odd zone we desire a negative frequency
    req_zone = int(np.ceil(abs(fc)/((block.BlockStatus['SamplingFreq']*1e3)/2)))
    if req_zone != zone:
        block.NyquistZone = req_zone
    if even:
        true_fc = fc if btype=='adc' else -fc
    else:
        true_fc = -fc if btype=='adc' else fc
    block.MixerSettings['Freq'] = true_fc
    block.UpdateEvent(1)

## Introducing the Coarse Mixer <a class="anchor" id="introducing_coarse_mixer"></a>

The coarse mixer will allow us to modulate and demodulate using specific frequencies which are simple to store using basic combinatorial logic, namely $\frac{f_{s}}{4}$, $-\frac{f_{s}}{4}$ and $\frac{f_{s}}{2}$. This makes the coarse mixer only suited for specific use cases, but it can operate at much lower power. 

We can set the tx_channel and rx_channel mixer settings per RF-DAC/RF-ADC block using *.MixerSettings.update*.

To make this demonstration interesting, we will transmit a signal from the RF-DAC and receive it using the RF-ADC. To this extent we will operate the RF-DAC in fine mixer mode and explore the different Coarse mixer options in the RF-ADC.

In [11]:
# Set DAC to Fine Mixer Type
tx_channel.dac_block.MixerSettings.update({
    'MixerType': xrfdc.MIXER_TYPE_FINE,
    'FineMixerScale': xrfdc.MIXER_SCALE_0P7,
})

### Coarse Mixer: fs/4 <a class="anchor" id="coarse_fs_4"></a>

First, we will explore what happens when the coarse mixer is set to $\frac{f_{s}}{4}$. We do this by updating the MixerType and CoarseMixFreq variables at the same time using *MixerSettings.update*.

In [12]:
# Set RF-ADC channel 0 to Coarse Mixer, fs/4
rx_channel.adc_block.MixerSettings.update({
    'CoarseMixFreq' : xrfdc.COARSE_MIX_SAMPLE_FREQ_BY_FOUR,
   'MixerType': xrfdc.MIXER_TYPE_COARSE,
})

**Time Domain Plot**

We can transmit a signal from the RF-DAC to the RF-ADC and plot the results in the time domain. We chose an RF-DAC modulation frequency initially at 800 MHz and transfer the data in packets to the receiver. 

In [13]:
# transmission carrier - feel free to change this and observe the results!
configure_mixer_frequency(block=tx_channel.dac_block, btype='dac', fc=800) 

In [14]:
# transmit data
c_data   = rx_channel.transfer(packetsize=number_samples)

We can see the result of this transmission in the time domain. We know:

$f_{s} = 4096$


$f_{c} = \frac{f_{s}}{4} = 1024$

Therefore, through hetereodyning we expect a mix of the RF-ADC $f_{c}$ at 1024 MHz and the RF-DAC $f_{c}$ at 800 MHz, producing a signal at 224 MHz.

In [15]:
# plot in time domain
plots.time_plot(number_samples, sample_frequency, c_data)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

**Frequency Domain**

We can now plot the same transmission in the frequency domain, observing the expected signal at 224 MHz.

In [16]:
# plot in fequency domain
plots.freq_plot(c_data, number_samples, sample_frequency)

FigureWidget({
    'data': [{'name': 'Spectrum',
              'type': 'scatter',
              'uid': 'e0a6e4…

### Coarse Mixer: -fs/4 <a class="anchor" id="course_min_fs_4"></a>

Now we explore what happens when the Coarse mixer is set to $\frac{-f_{s}}{4}$. We will leave the RF-DAC in Fine mixer mode, but change the CoarseMixFreq setting of the RF-ADC channel.

In [17]:
# Set RF-ADC channel 0 to Coarse Mixer, -fs/4
rx_channel.adc_block.MixerSettings.update({
    'CoarseMixFreq' : xrfdc.COARSE_MIX_MIN_SAMPLE_FREQ_BY_FOUR,
   'MixerType': xrfdc.MIXER_TYPE_COARSE,
})

**Time Domain Plot**

Once more, we can transmit a signal from the RF-DAC to the RF-ADC and plot the results in the time domain. We chose an RF-DAC modulation frequency initially at 800 MHz and transfer the data in packets to the receiver. 

In [18]:
# transmission carrier
configure_mixer_frequency(block=tx_channel.dac_block, btype='dac', fc=800) # transmission carrier

In [19]:
# transmit data
c_data = rx_channel.transfer(packetsize=number_samples)

We can see the result of this transmission in the time domain. Once more we know: 

$f_{s} = 4096$


$f_{c} = \frac{-f_{s}}{4} = -1024$

Therefore, through hetereodyning we expect a mix of the RF-ADC $f_{c}$ at -1024 MHz and the RF-DAC $f_{c}$ at 800 MHz, producing a signal at -224 MHz.

In [20]:
# plot in time domain
plots.time_plot(number_samples, sample_frequency, c_data)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

**Frequency Domain**

In [21]:
# plot in frequency domain
plots.freq_plot(c_data, number_samples, sample_frequency)

FigureWidget({
    'data': [{'name': 'Spectrum',
              'type': 'scatter',
              'uid': '2fa691…

### Coarse Mixer: fs/2 <a class="anchor" id="coarse_fs_2"></a>

The final operating mode of the coarse mixer is $\frac{f_{s}}{2}$. This mode can be set as shown below, once again using *MixerSettings.update*. We expect to see a 'flipping' of the input in both time and frequency domains. This can be useful in correcting the folding which occurs when 'even' Nyquist Zones are aliased to baseband. 

In [22]:
# set coarse mixer to fs/2
rx_channel.adc_block.MixerSettings.update({
    'CoarseMixFreq' : xrfdc.COARSE_MIX_SAMPLE_FREQ_BY_TWO,
   'MixerType': xrfdc.MIXER_TYPE_COARSE,
})

## Introducing the Fine Mixer <a class="anchor" id="introducing_fine_mixer"></a>

The fine mixer mode allows for precise control of the NCO, setting the mixer frequency between $-\frac{f_{s}}{2}$ and $\frac{f_{s}}{2}$. 

In [27]:
# set RF-DAC channel 0 to fine mixer
tx_channel.dac_block.MixerSettings.update({
    'MixerType': xrfdc.MIXER_TYPE_FINE,
    'FineMixerScale': xrfdc.MIXER_SCALE_0P7,
})

In [28]:
# set RF-ADC channel 0 to fine mixer
rx_channel.adc_block.MixerSettings.update({
    'MixerType': xrfdc.MIXER_TYPE_FINE,
    'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
})

**Time Domain**

We are now able to freely configure a mixer frequency for the RF-ADC and RF-DAC channels, using the *configure_mixer_frequency* function.

In [29]:
# configure transmit and receive centre frequencies
configure_mixer_frequency(block=tx_channel.dac_block, btype='dac', fc=800) # transmission carrier
configure_mixer_frequency(block=rx_channel.adc_block, btype='adc', fc=-1200) # receiver carrier

In [30]:
# transmit data
c_data   = rx_channel.transfer(packetsize=number_samples)

Once again we can observe the effects of hetereodyning, by mixing signals at 800 MHz and -1200 MHz we produce a received signal at 400 MHz.

In [31]:
# plot in time domain
plots.time_plot(number_samples, sample_frequency, c_data)

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

**Frequency Domain**

In [32]:
# plot in frequency domain
plots.freq_plot(c_data, number_samples, sample_frequency)

FigureWidget({
    'data': [{'name': 'Spectrum',
              'type': 'scatter',
              'uid': 'd983b6…

## Mixer Bypass <a class="anchor" id="mixer_bypass"></a>

Mixer bypass mode allows us to ignore the modulation process and directly transmit our baseband signal. As you can imagine this would be considerably more inefficient than utalising a Radio Frequency carrier as we have been up to now.

However there are cases where this may have uses, such as when we are interested in an intermediate step in the transmission chain, moving baseband to an Intermediate Frequency (IF) transceiver and then to RF.

Bypass mode can be set as shown below. As we explored previously the circuitry to set bypass mode is within the coarse mixer, so we can choose the BYPASS setting as we would any of the frequency options for the coarse mixer.

In [33]:
# configure RF-ADC channel 0 for mixer bypass
rx_channel.adc_block.MixerSettings.update({
    'CoarseMixFreq' : xrfdc.COARSE_MIX_BYPASS,
   'MixerType': xrfdc.MIXER_TYPE_COARSE,
})

## Conclusion <a class="anchor" id="conclusion"></a>

This notebook has presented a simple introduction to the Mixer Settings available using the RF Data Converters of the RFSoC2x2. We have explored the Coarse and Fine mixer modes in a design running on the RFSoC2x2 board, as well as exploring the theory of Bypass mode.

RF data visualization was performed using `plotly`, `ipywidgets`, and `numpy` in Jupyter.

You can also return to the [Coarse Mixer](#introducing_coarse_mixer) or [Fine Mixer](#introducing_fine_mixer) sections and try different mixer frequencies.

[⬅️ Previous Notebook](rfsoc_fpga_interface.ipynb) | | [Next Notebook ➡️](rfsoc_multirate.ipynb)

----
----